In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import display

band = widgets.IntSlider(
    value=3,
    min=1,
    max=8,
    step=1,
    description='Number of Band',
)

row = widgets.IntSlider(
    value=64,
    min=20,
    max=128,
    step=1,
    description='Number of row in a band',
)

sigma = widgets.FloatSlider(
    value=2.0,
    min=1.0,
    max=10.0,
    step=0.5,
    description='Value of sigma',
)

feature_list = widgets.SelectMultiple(
    description="Feature List",
    value=('duration', 'end_of_fade_in'),
    options=['duration', 'end_of_fade_in', 'key', 'loudness', 'mode', 'start_of_fade_out', 'tempo', 'time_signature']
)

display(band)
display(row)
display(sigma)
display(feature_list)

button = widgets.Button(description="Execute!")
display(button)

def on_button_clicked(b):
    clear_output()
    print("Let me check the parameters...")
    if not feature_list.value:
        print("Oops!! feature list cannont be empty.Please recheck.")
        print("Program Exiting...")
    else:
        print("Parameters looks to be okay. I will call backend now..")
        back_end_call(band.value, row.value, sigma.value, feature_list.value)

button.on_click(on_button_clicked)

def back_end_call(num_band, num_row, sigma, feature_list):
    print(num_band)
    print(num_row)
    print(sigma)
    print(feature_list)